In [74]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv
import os
import math
import pandas
import scipy

In [75]:
'''reading the data from the file and getting the values of X and Y'''
os.chdir('/Users/sreelakshmirajendrakumar/Downloads')
datasetLasso = pandas.read_csv('project3_F2019_lasso.csv')
XLasso=datasetLasso.iloc[:,0:251]
XLasso=XLasso.to_numpy()
#appending the missing row from the dataset
firstValues=list(datasetLasso.columns.values)
for i in range(0,251):
    firstValues[i]=float(firstValues[i])
valuesToAppend=np.zeros((251,1))
firstValues=np.array(firstValues)
for i in range(0,251):
    valuesToAppend[i]=firstValues[i]
XLasso=np.concatenate((valuesToAppend.T,XLasso),axis=0) # is 200 the number of samples and 250 the number of dimensions?

X=XLasso[:,0:-1] # shape(250,200)
Y=XLasso[:,-1] # shape(200,)

In [76]:
def meanColumn(a):
    '''finding the mean of X to centralize'''
    (row,column)=np.shape(a)
    mean=[]
    for i in range (0,column):
        temp=0
        for j in range(0,row):
            temp+=a[j][i]
        mean.append(temp/row)
    return mean #1xcolumn

def sdColumn(a):
    '''finding the sd of X to centralize'''
    (row,column)=np.shape(a)
    sd=[]
    for j in range(0,column):
        sd.append(np.std(np.array(a)[:,j]))
    return sd #1xcolumn

def centralizeColumn(a): # input: X
    '''centralizing the values in a matrix'''
    (row,column)=np.shape(a)
    mean=meanColumn(a)
    sd=sdColumn(a)
    newMean=[]
    newSD=[]
    for i in range(0,row):
        newMean.append(mean)
        newSD.append(sd)
    centralized=np.zeros((row,column))
    for i in range(0,row):
        for j in range(0,column):
            centralized[i][j]=((a[i][j]-newMean[i][j])/newSD[i][j])
    return centralized

def attach(attachX): # input: centralized X
    '''attaching ones to the first column'''
    (N,d)=np.shape(attachX)
    tempX=np.ones((N,1))
    attachedX=np.concatenate((tempX,attachX),axis=1)
    return attachedX

theLambda=np.arange(1,301,1)
theLambda=np.square(theLambda)

(N,d)=np.shape(X)
X=centralizeColumn(X)
X=attach(X)

In [77]:
def shrinkageOperator(shrinkV,shrinkLambda,shrinkRho):
    '''defining the shrinkage operator'''
    if shrinkV>=shrinkLambda*shrinkRho:
        shrinkValue=shrinkV-(shrinkLambda*shrinkRho)
    elif shrinkV==shrinkLambda*shrinkRho:
        shrinkValue=0
    elif shrinkV<=shrinkLambda*shrinkRho:
        shrinkValue=shrinkV+(shrinkLambda*shrinkRho)
    return shrinkValue


In [96]:
def ADMM(admmGamma,admmTau,admmLambda,admmRho,admmD,N=N,d=d,admmX=X,admmY=Y):
    '''defining beta'''
    toBeInversed=np.matmul(admmX.T,admmX)+(np.multiply((1/(2*admmRho)),np.matmul(admmD.T,admmD))) # (d+1)x(d+1)
    toMultiply_beta=np.linalg.inv(toBeInversed) #(d+1)x(d+1)
    subtracted=np.subtract(admmGamma,admmTau) #(d+1)x1
    toMultiplyAgain_beta=np.matmul(admmX.T,admmY)+np.multiply((1/(2*admmRho)),(np.matmul(admmD.T,subtracted)))#(d+1)x1
    newBeta=np.matmul(toMultiply_beta,toMultiplyAgain_beta) #(d+1)x1
    
    '''defining gamma'''
    DB=np.matmul(admmD,newBeta)#(d+1)x1
    DB=np.reshape(DB,(d+1,1))
    newGamma=np.zeros((d+1),1)
    newGamma=np.reshape(newGamma,(d+1,d+1))
    for i in range(0,d+1):
        shrinkageInput=DB[i]+admmTau[i]
        newGamma[i]=shrinkageOperator(shrinkageInput,admmLambda,admmRho)
        
    '''defining tau'''
    newTau=DB
    newTau=np.subtract(np.add(admmTau,DB),newGamma)
    
    return newBeta,newGamma,newTau
hh=ADMM(np.random.rand(d+1,1),np.random.rand(d+1,1),3,2,np.random.rand(d+1,d+1))

ValueError: cannot reshape array of size 63001 into shape (251,1)

In [89]:
def ADMMiteration(iterLambda,iterRho,iterD,N=N,d=d,iterNum=100,iterX=X,iterY=Y):
    flag=0
    allTau=[]
    allBeta=[]
    allGamma=[]

    initialTau=np.random.rand((d+1),1)
    initialGamma=np.random.rand((d+1),1)
    initialBeta,e,w=ADMM(initialGamma,initialTau,iterLambda,iterRho,iterD)
    
    allTau.append(initialTau)
    allGamma.append(initialGamma)
    allBeta.append(initialBeta)
 
    '''i=1
    while True:
        tempBeta,tempGamma,tempTau=ADMM(allGamma[i-1],allTau[i-1],iterLambda,iterRho,iterD)
        tBeta=np.reshape(np.array(tempBeta),(d+1,1))
        tGamma=np.reshape(np.array(tempGamma),(d+1,1))
        tTau=np.reshape(np.array(tempTau),(d+1,1))
        allTau.append(tTau)
        allGamma.append(tGamma)
        allBeta.append(tBeta)


        errorTau=np.abs(np.subtract(allTau[i],allTau[i-1]))
        errorBeta=np.abs(np.subtract(allBeta[i],allBeta[i-1]))
        errorGamma=np.abs(np.subtract(allGamma[i],allGamma[i-1]))
        
        i+=1
        
        #if any(errorTau)<10**(-6) or any(errorGamma)<10**(-6) or any(errorBeta)<10**(-6):
        #    flag=0
        #else:
        #    flag=1
            
        if i==iterNum:
            break'''
        
    return allBeta[1:],allGamma[1:],allTau[1:]

    
iteratedBeta,iteratedGamma,iteratedTau=ADMMiteration(3,2,np.random.rand((d+1),(d+1)))   

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
def truncateBeta(truncBeta): # pass the value of beta obtained on last the iteration
    for i in range(len(truncBeta)):
        if abs(truncBeta[i])<10**(-5):
            truncBeta[i]=0
    return truncBeta 




In [ ]:
def returnCoordinates(returnBeta):
    coordinates=[]
    for j in range(0,len(returnBeta)):
        if returnBeta[j]!=0:
            coordinates.append(j)
    print ('the non-vanishing coordinates are:\n' )
    print (coordinates)
    return coordinates

meanForY=np.multiply(X.T,iteratedBeta[-1]) 

'''before proceeding next, define theLambda and find the values of beta for all those lambda values''' 

In [ ]:
def separate(Beta): # input: beta
    BetaZero=Beta[0]
    B=Beta[1:-1]
    
    return BetaZero,B  

In [ ]:
lambdaBeta=[]
lambdaBetaZero=[]
lambdaB=[]
for index,value in enumerate(theLambda):
    rho=1/value # choosing rho value that is inverse to lambda
    temp,a,b=ADMMiteration(X,Y,value,rho,np.random.rand((d+1),(d+1)))
    temp1=truncateBeta(temp[-1]) # use the last iterated value by the function
    lambdaBeta.append(temp1) 
    
    '''separating beta and b'''
    temp1,temp2=separate(lambdaBeta[index])
    lambdaBetaZero.append(temp1)
    lambdaB.append(temp2)

In [ ]:
'''plots for the question'''
def nonVanishingCoeff(recoverLambdaB=lambdaB): # input: the big array which has the b values for all lambda values
    toPlot=[]
    for k in range(0,len(recoverLambdaB)):
        w=0
        for j in range(0,len(recoverLambdaB[0])):
            if recoverLambdaB[k][j]!=0:
                w+=1
    toPlot.append(w)   #output: number of non-vanishing elements in b==> to plot against lambda 
    return toPlot
    
def plotB(jValue,LambdaB=lambdaB): # input: the big array which has b values for all lambda values and j=21,36,200
    toBePlotted=[]
    for k in range(0,len(LambdaB)):
        toBePlotted.append(LambdaB[k][j]) # plot against lambda
        
    return toBePlotted

def increment(j,LambdaB=lambdaB,TheLambda=theLambda): # input: the big array of b and j=21,36,200
    toPlot=[]
    for k in range(0,len(LambdaB)-1):
        temp=LambdaB[k+1][j]-Lambda[k][j]
        temp1=theLambda[k+1]-thelambda[k]
        temp3=temp/temp1
        toPlot.append(temp3) # note that this is one element less than that of theLambda

In [ ]:
'''Plotting'''
w=nonVanishingCoeff()
plt.plot(theLambda,w)
plt.xlabel('lambda')
plt.ylable('Number of non-vanishing coefficients')
plt.title('Plotting non vanishing coefficients in b corresponding to the lambda values')


"""plot of b(j) correspomding to each lambda"""
j21=plotB(21)
plt.plot(theLambda,j21)
plt.xlabel('lambda')
plt.ylable('Value of b(j) when j=21')
plt.title('Plotting the value of b(j) corresponding to the lambda values when j=21')

j36=plotB(36)
plt.plot(theLambda,j36)
plt.xlabel('lambda')
plt.ylable('Value of b(j) when j=36')
plt.title('Plotting the value of b(j) corresponding to the lambda values when j=36')

j200=plotB(200)
plt.plot(theLambda,j200)
plt.xlabel('lambda')
plt.ylable('Value of b(j) when j=200')
plt.title('Plotting the value of b(j) corresponding to the lambda values when j=200')

'''plot of increment corresponding to each lambda'''
J21=increment(21)
plt.plot(theLambda[1:],J21)
plt.xlabel('lambda')
plt.ylable('Value of increment when j=21')
plt.title('Plotting the value of increment corresponding to the lambda values when j=21')

J36=plotB(36)
plt.plot(theLambda[1:],J36)
plt.xlabel('lambda')
plt.ylable('Value of b(j) when j=36')
plt.title('Plotting the value of increment corresponding to the lambda values when j=36')

J200=plotB(200)
plt.plot(theLambda[1:],J200)
plt.xlabel('lambda')
plt.ylable('Value of increment when j=200')
plt.title('Plotting the value of increment corresponding to the lambda values when j=200')




